In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm 
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats


# In[42]:


split = (0.8);
sequence_length=10;
normalise= True
batch_size=100;
input_dim=1
input_timesteps=9
neurons=50
epochs=50
prediction_len=1
dense_output=1
drop_out=0

dataframe = pd.read_csv("D:\DeTaiNam2024\SoLieuHong\KDC.csv")
cols = ['close']

len_dataframe=dataframe.shape[0]

i_split = int(len(dataframe) * split)
data_train = dataframe.get(cols).values[:i_split]
data_test  = dataframe.get(cols).values[i_split:]
len_train  = len(data_train)
len_test   = len(data_test)
len_train_windows = None
print('data_train.shape',data_train.shape)
print('data_test.shape',data_test.shape)

#get_test_data    
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
 # get original y_test
y_test_ori = data_windows[:, -1, [0]]
print('y_test_ori.shape',y_test_ori.shape)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]
normalised_data = []
record_min=[]
record_max=[]

#normalize
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data#get_test_data
x_test = data_windows[:, :-1]
y_test = data_windows[:, -1, [0]]
print('x_test.shape',x_test.shape)
print('y_test.shape',y_test.shape)

#get_train_data 
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
  
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train = data_windows[:, :-1]
y_train = data_windows[:, -1,[0]]
print('x_train.shape',x_train.shape)
print('y_train.shape',y_train.shape)


# In[43]:


# LSTM MODEL
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='relu'))
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)


# model.predict(x_test)

# In[44]:


model.predict(x_test)


# In[46]:


#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)
    
#print(prediction_seqs)


# In[48]:


#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
#print(de_predicted)


# In[50]:


error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])
    
squaredError = []
absError = []
for val in error:
    squaredError.append(val * val) 
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)



data_train.shape (1940, 1)
data_test.shape (486, 1)
y_test_ori.shape (476, 1)
x_test.shape (476, 9, 1)
y_test.shape (476, 1)
x_train.shape (1930, 9, 1)
y_train.shape (1930, 1)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9804\2452096333.py:91: RuntimeWarning: invalid value encountered in divide
  temp_col=temp_col/temp_max


Epoch 1/50
20/20 [==============================] - 8s 32ms/step - loss: 0.3709
Epoch 2/50
20/20 [==============================] - 0s 25ms/step - loss: 0.3709
Epoch 3/50
20/20 [==============================] - 1s 27ms/step - loss: 0.3709
Epoch 4/50
20/20 [==============================] - 1s 25ms/step - loss: 0.3709
Epoch 5/50
20/20 [==============================] - 1s 25ms/step - loss: 0.3709
Epoch 6/50
20/20 [==============================] - 0s 25ms/step - loss: 0.3709
Epoch 7/50
20/20 [==============================] - 1s 25ms/step - loss: 0.3709
Epoch 8/50
20/20 [==============================] - 0s 25ms/step - loss: 0.3709
Epoch 9/50
20/20 [==============================] - 0s 24ms/step - loss: 0.3709
Epoch 10/50
20/20 [==============================] - 0s 25ms/step - loss: 0.3709
Epoch 11/50
20/20 [==============================] - 1s 25ms/step - loss: 0.3709
Epoch 12/50
20/20 [==============================] - 0s 24ms/step - loss: 0.3709
Epoch 13/50
20/20 [==================

In [7]:
# Đọc dữ liệu
data = pd.read_csv("D:\DeTaiNam2024\SoLieuHong\KDC.csv")
data['time'] = pd.to_datetime(data['time'])
data.set_index('time', inplace=True)

# Chọn cột giá đóng cửa
close_prices = data['close']
train_size = int(len(close_prices) * 0.8)
train, test = close_prices[:train_size], close_prices[train_size:]

C:\Users\Admin\AppData\Local\Temp\ipykernel_9804\3658374132.py:3: UserWarning: Parsing '13/10/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  data['time'] = pd.to_datetime(data['time'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_9804\3658374132.py:3: UserWarning: Parsing '14/10/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  data['time'] = pd.to_datetime(data['time'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_9804\3658374132.py:3: UserWarning: Parsing '15/10/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  data['time'] = pd.to_datetime(data['time'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_9804\3658374132.py:3: UserWarning: Parsing '16/10/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  data['time'] = pd.to_datetime(data['time'])
C:\Users\Admin\AppDa

In [8]:
from sklearn.metrics import mean_squared_error
train_lstm = np.array(train).reshape(-1, 1)
model_lstm = Sequential([
    LSTM(50, activation='relu', input_shape=(1, 1)),
    Dense(1)
])
model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.fit(train_lstm[:-1].reshape(-1, 1, 1), train_lstm[1:], epochs=200, verbose=0)
lstm_predictions = model_lstm.predict(np.array(test).reshape(-1, 1, 1)).flatten()
lstm_mse = mean_squared_error(test, lstm_predictions)

16/16 [==============================] - 0s 2ms/step


In [9]:
mse1=np.mean(np.square(lstm_predictions - test))
print('MSE=', mse1)
mean_error_percent1 = np.mean(np.abs((lstm_predictions - test) / test))
accuracy1 = 1 - mean_error_percent1
print('Accuracy=', accuracy1)
print('mean_error_percent=', mean_error_percent1)

MSE= 0.05605671948908161
Accuracy= 0.9962034705962759
mean_error_percent= 0.0037965294037240567
